# Music Generation with LSTM

In [1]:
import glob
import numpy as np
import pickle
from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

In [2]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./full_set_beethoven_mozart directory. Call BEFORE train """
    notes = []
    durations = []
    files = "chopin/*.mid"

    for file in glob.glob(files):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi) #Change to only grab the piano???
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    with open('data/notes.p', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [3]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [4]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    units = 512
    dropout = 0.3
    
    model = Sequential()
    model.add(
        Bidirectional(
            LSTM(
                units,
                dropout=dropout,
                return_sequences=True
            ),
            input_shape=(network_input.shape[1], network_input.shape[2])
        )
    )
    model.add(Dropout(dropout))
    model.add(SeqSelfAttention(attention_activation='relu'))
    model.add(LSTM(units, dropout=dropout))
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(clipnorm=1.0))
    
    model.summary()

    return model

In [6]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        'weights.hdf5',
        monitor='loss',
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(
        x=network_input,
        y=network_output,
        batch_size=1024,
        epochs=2000,
        callbacks=callbacks_list
    )

In [7]:
train_network()

Parsing chopin/chp_op18_format0.mid
Parsing chopin/chpn_op27_2.mid
Parsing chopin/chpn_op25_e1.mid
Parsing chopin/chpn-p19.mid
Parsing chopin/chpn-p19_format0.mid
Parsing chopin/chpn-p5.mid
Parsing chopin/chpn_op25_e1_format0.mid
Parsing chopin/chpn_op7_1.mid
Parsing chopin/chpn_op27_1.mid
Parsing chopin/chpn_op10_e01_format0.mid
Parsing chopin/chpn-p20_format0.mid
Parsing chopin/chpn_op25_e3.mid
Parsing chopin/chpn_op10_e05_format0.mid
Parsing chopin/chpn-p24_format0.mid
Parsing chopin/chpn_op25_e12_format0.mid
Parsing chopin/chpn-p24.mid
Parsing chopin/chpn-p6_format0.mid
Parsing chopin/chpn-p6.mid
Parsing chopin/chpn-p9_format0.mid
Parsing chopin/chpn_op33_2_format0.mid
Parsing chopin/chpn-p2.mid
Parsing chopin/chpn-p10.mid
Parsing chopin/chp_op31.mid
Parsing chopin/chpn-p7.mid
Parsing chopin/chpn_op35_3_format0.mid
Parsing chopin/chpn-p14_format0.mid
Parsing chopin/chpn-p17_format0.mid
Parsing chopin/chpn_op53_format0.mid
Parsing chopin/chpn_op25_e3_format0.mid
Parsing chopin/chpn-

101/101 [==============================] - 76s 748ms/step - loss: 3.7724
Epoch 46/2000
101/101 [==============================] - 75s 742ms/step - loss: 3.7098
Epoch 47/2000
101/101 [==============================] - 76s 754ms/step - loss: 3.6578
Epoch 48/2000
101/101 [==============================] - 75s 744ms/step - loss: 3.6058
Epoch 49/2000
101/101 [==============================] - 76s 756ms/step - loss: 3.5487
Epoch 50/2000
101/101 [==============================] - 75s 743ms/step - loss: 3.5003
Epoch 51/2000
101/101 [==============================] - 75s 747ms/step - loss: 3.4485
Epoch 52/2000
101/101 [==============================] - 76s 749ms/step - loss: 3.3930
Epoch 53/2000
101/101 [==============================] - 75s 744ms/step - loss: 3.3568
Epoch 54/2000
101/101 [==============================] - 75s 742ms/step - loss: 3.3013
Epoch 55/2000
101/101 [==============================] - 75s 745ms/step - loss: 3.2470
Epoch 56/2000
101/101 [==============================] - 

101/101 [==============================] - 76s 749ms/step - loss: 1.6073
Epoch 139/2000
101/101 [==============================] - 76s 753ms/step - loss: 1.5974
Epoch 140/2000
101/101 [==============================] - 76s 755ms/step - loss: 2.4320
Epoch 141/2000
101/101 [==============================] - 76s 757ms/step - loss: 1.5572
Epoch 142/2000
101/101 [==============================] - 76s 754ms/step - loss: 1.5439
Epoch 143/2000
101/101 [==============================] - 77s 762ms/step - loss: 1.5276
Epoch 144/2000
101/101 [==============================] - 77s 758ms/step - loss: 1.5277
Epoch 145/2000
101/101 [==============================] - 77s 758ms/step - loss: 2.0269
Epoch 146/2000
101/101 [==============================] - 76s 751ms/step - loss: 1.5416
Epoch 147/2000
101/101 [==============================] - 77s 760ms/step - loss: 1.5000
Epoch 148/2000
101/101 [==============================] - 76s 753ms/step - loss: 1.5033
Epoch 149/2000
101/101 [=======================

KeyboardInterrupt: 